In [30]:
from transformers import AutoConfig, AutoModel, AutoTokenizer
import os
import torch

os.environ["CUDA_VISIBLE_DEVICES"] = "2"
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))

d1='/data/yanghq/models/THUDM/chatglm-6b'
d2='/data/yanghq/outputs/pt-4b-128-2e-2/checkpoint-3000'

d1='/data/yanghq/models/YHQ/chatglm-6b'
d2='/data/yanghq/outputs/pt-4b-128-2e-2/checkpoint-500'

d1='/data/yanghq/outputs/ft-mode_0}-lr_1e-5/checkpoint-100'
d2=d1

tokenizer = AutoTokenizer.from_pretrained(d1, trust_remote_code=True)
config = AutoConfig.from_pretrained(d1, trust_remote_code=True, pre_seq_len=128)
config.ptoken=0
print(config)


Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


1
NVIDIA GeForce RTX 3090


Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.


ChatGLMConfig {
  "_name_or_path": "/data/yanghq/outputs/ft-mode_0}-lr_1e-5/checkpoint-100",
  "architectures": [
    "ChatGLMForConditionalGeneration"
  ],
  "auto_map": {
    "AutoConfig": "configuration_chatglm.ChatGLMConfig",
    "AutoModel": "modeling_chatglm.ChatGLMForConditionalGeneration",
    "AutoModelForSeq2SeqLM": "modeling_chatglm.ChatGLMForConditionalGeneration"
  },
  "bos_token_id": 130004,
  "eos_token_id": 130005,
  "gmask_token_id": 130001,
  "hidden_size": 4096,
  "inner_hidden_size": 16384,
  "layernorm_epsilon": 1e-05,
  "mask_token_id": 130000,
  "max_sequence_length": 2048,
  "model_type": "chatglm",
  "num_attention_heads": 32,
  "num_layers": 28,
  "pad_token_id": 3,
  "position_encoding_2d": true,
  "pre_seq_len": 128,
  "prefix_projection": false,
  "ptoken": 0,
  "quantization_bit": 0,
  "torch_dtype": "float16",
  "transformers_version": "4.27.1",
  "use_cache": true,
  "vocab_size": 130528
}



In [28]:
model = AutoModel.from_pretrained(d1, config=config, trust_remote_code=True).quantize(8).half().cuda()
print(model)
'''
prefix_state_dict = torch.load(os.path.join(d2, "pytorch_model.bin"))
new_prefix_state_dict = {}
for k, v in prefix_state_dict.items():
    if k.startswith("transformer.prefix_encoder."):
        new_prefix_state_dict[k[len("transformer.prefix_encoder."):]] = v
model.transformer.prefix_encoder.load_state_dict(new_prefix_state_dict)
'''

Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.
Could not locate the modeling_chatglm.py inside /data/yanghq/outputs/ft-mode_0}-lr_1e-5/checkpoint-100/global_step1600.


OSError: /data/yanghq/outputs/ft-mode_0}-lr_1e-5/checkpoint-100/global_step1600 does not appear to have a file named modeling_chatglm.py. Checkout 'https://huggingface.co//data/yanghq/outputs/ft-mode_0}-lr_1e-5/checkpoint-100/global_step1600/None' for available files.

In [25]:
response, history = model.chat(tokenizer, "类型#上衣*材质#牛仔布*颜色#白色*风格#简约*图案#刺绣*衣样式#外套*衣款式#破洞", history=[])
print(response)

In [26]:
len(response)

0

In [1]:
import torch

In [8]:
def distribute_insert_tensor(insert_tensor, large_tensor,insert_tensor_2,large_tensor_2, dim=0):
    d = insert_tensor.device
    large_length = large_tensor.size(dim)
    insert_length = insert_tensor.size(dim)
    total_length = large_length + insert_length
    result_size = insert_tensor.shape
    result_size = torch.Size([total_length])+result_size[1:]
    
    indices_insert = torch.randperm(total_length, device=d)[:(insert_length)]
    indices_insert = indices_insert.sort()[0]
    
    indices_large = torch.ones(total_length, dtype=torch.bool, device=d)
    indices_large[indices_insert] = False
    result_tensor = torch.empty(result_size, dtype=insert_tensor.dtype, device=d)
    result_tensor = result_tensor.contiguous()
    result_tensor[indices_insert] = insert_tensor
    result_tensor[indices_large] = large_tensor
    result_tensor_2 = torch.empty(result_size, dtype=insert_tensor.dtype, device=d)
    result_tensor_2 = result_tensor_2.contiguous()
    result_tensor_2[indices_insert] = insert_tensor_2
    result_tensor_2[indices_large] = large_tensor_2
    #print(insert_tensor.shape,large_tensor.shape,result_tensor.shape)
    return result_tensor,result_tensor_2

In [10]:
a=torch.rand([4,6,8])
c=torch.rand([4,6,8])
b=torch.rand([12,6,8])
d=torch.rand([12,6,8])

e,f = distribute_insert_tensor(a,b,c,d)
print(e.shape,f.shape)

torch.Size([16, 6, 8]) torch.Size([16, 6, 8])
